***Import Lib***

In [1]:
#!conda install -c conda-forge geocoder --yes 
import geocoder # import geocoder
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup

print('Libraries imported.')

Libraries imported.


In [2]:
!wget -q -O 'toronto_data.txt' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M
print('Data downloaded!')

Data downloaded!


***Read File***

In [3]:

with open('toronto_data.txt') as file_data:
    toronto_list_data = file_data.readlines()

***Select only Toronto table Data***

In [4]:
toronto_html_data = ''.join(toronto_list_data)

startIndex = toronto_html_data.find("<table")
endIndex = toronto_html_data.find("</table>")


toronto_html_data = toronto_html_data[startIndex:endIndex].replace("\n","")                 
                                  

***Use BeautifilSoup to parse html data to table***

In [5]:

table_data = [[cell.text for cell in row("td")]
                         for row in BeautifulSoup(toronto_html_data, "lxml")("tr")]


In [6]:
table_data[0:5]

[[],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

***Create DataFrame***

In [7]:
# define the dataframe columns
column_names = ['PostalCode','Borough', 'Neighborhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

***Fill Data into DataFrame***

In [8]:

for data in table_data:
    
    #check data exist and Borough is assigned
    if data and data[1] != "Not assigned":
        PostalCode = data[0]
        borough = data[1] 
        
        #check Neighborhood is assigned
        if data[2] == "Not assigned":
            neighborhood_name = data[1]
        else:    
            neighborhood_name = data[2]

        neighborhoods = neighborhoods.append({'PostalCode': PostalCode,
                                              'Borough': borough,
                                              'Neighborhood': neighborhood_name}, ignore_index=True)

In [9]:
neighborhoods.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [10]:
neighborhoods.shape

(212, 3)

***Get Geo Location***

In [11]:

def getLocation(postal_code):
    # initialize your variable to None
    count = 0;
    lat_lng_coords = None

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng
      count += 1
      if(count%50 == 0):
          print(count)

    return lat_lng_coords


Because of long run loop.

So I decide to use "Geospatial_Coordinates.csv" 

In [12]:
!wget -q -O 'Geospatial_Coordinates.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
geoLocationDF = pd.read_csv("Geospatial_Coordinates.csv")
geoLocationDF.set_index('Postal Code', inplace=True)

In [14]:
def getLocationFromDF(row):
    return geoLocationDF.loc[row['PostalCode']]

***Apply function to get Location***

In [15]:
locationApply = neighborhoods.apply(getLocationFromDF, axis=1)
neighborhoods = neighborhoods.join(locationApply)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763


In [16]:
neighborhoods.shape

(212, 5)